In [201]:
from src.mazeData import *
from src.maze2025GraphClass import mazeGraph
from pyvis.network import Network
from src.PS_agentPrograms import *
from src.mazeProblemClass import MazeProblem
from src.agents import *
from src.naigationEnvironmentClass import MazeNavigationEnvironment
from src.Lab4Environment import *

In [202]:
def makeMaze(n):
  size = (n,n)
  proba_0 =0.25 # resulting array will have 25%? of zeros
  proba_Enemy =0.1 # resulting array will have 10%? of enemy
  arrMaze=np.random.choice([0, 1, 2], size=size, p=[proba_0, 1-proba_0-proba_Enemy, proba_Enemy] )
  return arrMaze

initState = (random.randint(0,6),random.randint(0,6))
goalState = (random.randint(0,6),random.randint(0,6))

mazeSize=7
mainMaze = makeMaze(mazeSize)
MazeCheck(mainMaze,initState,goalState)
mazeAvalActs=defineMazeAvailableActions(mainMaze)
maze1TM=makeMazeTransformationModel(mazeAvalActs)
mazeWorldGraph=mazeGraph(maze1TM, mazeStatesLocations(list(maze1TM.keys())))

maze1TM

{(0, 0): {'down': (1, 0)},
 (0, 1): {},
 (0, 2): {},
 (0, 3): {'down': (1, 3)},
 (0, 4): {},
 (0, 5): {},
 (0, 6): {},
 (1, 0): {'up': (0, 0), 'down': (2, 0)},
 (1, 1): {},
 (1, 2): {'right': (1, 3)},
 (1, 3): {'left': (1, 2), 'up': (0, 3), 'right': (1, 4), 'down': (2, 3)},
 (1, 4): {'left': (1, 3), 'right': (1, 5), 'down': (2, 4)},
 (1, 5): {'left': (1, 4), 'right': (1, 6), 'down': (2, 5)},
 (1, 6): {'left': (1, 5)},
 (2, 0): {'up': (1, 0), 'right': (2, 1), 'down': (3, 0)},
 (2, 1): {'left': (2, 0)},
 (2, 2): {},
 (2, 3): {'up': (1, 3), 'right': (2, 4)},
 (2, 4): {'left': (2, 3), 'up': (1, 4), 'right': (2, 5), 'down': (3, 4)},
 (2, 5): {'left': (2, 4), 'up': (1, 5), 'down': (3, 5)},
 (2, 6): {},
 (3, 0): {'up': (2, 0), 'down': (4, 0)},
 (3, 1): {},
 (3, 2): {'down': (4, 2)},
 (3, 3): {},
 (3, 4): {'up': (2, 4), 'right': (3, 5), 'down': (4, 4)},
 (3, 5): {'left': (3, 4), 'up': (2, 5), 'right': (3, 6), 'down': (4, 5)},
 (3, 6): {'left': (3, 5)},
 (4, 0): {'up': (3, 0), 'down': (5, 0)},


In [203]:
mainMaze

array([[1, 0, 0, 1, 0, 0, 0],
       [1, 0, 1, 1, 2, 1, 1],
       [1, 1, 0, 1, 1, 1, 0],
       [1, 0, 1, 0, 1, 1, 1],
       [1, 0, 1, 1, 1, 1, 0],
       [1, 1, 1, 0, 1, 1, 1],
       [1, 1, 0, 0, 1, 1, 2]])

In [204]:
maze1TM

{(0, 0): {'down': (1, 0)},
 (0, 1): {},
 (0, 2): {},
 (0, 3): {'down': (1, 3)},
 (0, 4): {},
 (0, 5): {},
 (0, 6): {},
 (1, 0): {'up': (0, 0), 'down': (2, 0)},
 (1, 1): {},
 (1, 2): {'right': (1, 3)},
 (1, 3): {'left': (1, 2), 'up': (0, 3), 'right': (1, 4), 'down': (2, 3)},
 (1, 4): {'left': (1, 3), 'right': (1, 5), 'down': (2, 4)},
 (1, 5): {'left': (1, 4), 'right': (1, 6), 'down': (2, 5)},
 (1, 6): {'left': (1, 5)},
 (2, 0): {'up': (1, 0), 'right': (2, 1), 'down': (3, 0)},
 (2, 1): {'left': (2, 0)},
 (2, 2): {},
 (2, 3): {'up': (1, 3), 'right': (2, 4)},
 (2, 4): {'left': (2, 3), 'up': (1, 4), 'right': (2, 5), 'down': (3, 4)},
 (2, 5): {'left': (2, 4), 'up': (1, 5), 'down': (3, 5)},
 (2, 6): {},
 (3, 0): {'up': (2, 0), 'down': (4, 0)},
 (3, 1): {},
 (3, 2): {'down': (4, 2)},
 (3, 3): {},
 (3, 4): {'up': (2, 4), 'right': (3, 5), 'down': (4, 4)},
 (3, 5): {'left': (3, 4), 'up': (2, 5), 'right': (3, 6), 'down': (4, 5)},
 (3, 6): {'left': (3, 5)},
 (4, 0): {'up': (3, 0), 'down': (5, 0)},


In [205]:
mazeAvalActs

{(0, 0): ['down'],
 (0, 1): [],
 (0, 2): [],
 (0, 3): ['down'],
 (0, 4): [],
 (0, 5): [],
 (0, 6): [],
 (1, 0): ['up', 'down'],
 (1, 1): [],
 (1, 2): ['right'],
 (1, 3): ['left', 'up', 'right', 'down'],
 (1, 4): ['left', 'right', 'down'],
 (1, 5): ['left', 'right', 'down'],
 (1, 6): ['left'],
 (2, 0): ['up', 'right', 'down'],
 (2, 1): ['left'],
 (2, 2): [],
 (2, 3): ['up', 'right'],
 (2, 4): ['left', 'up', 'right', 'down'],
 (2, 5): ['left', 'up', 'down'],
 (2, 6): [],
 (3, 0): ['up', 'down'],
 (3, 1): [],
 (3, 2): ['down'],
 (3, 3): [],
 (3, 4): ['up', 'right', 'down'],
 (3, 5): ['left', 'up', 'right', 'down'],
 (3, 6): ['left'],
 (4, 0): ['up', 'down'],
 (4, 1): [],
 (4, 2): ['up', 'right', 'down'],
 (4, 3): ['left', 'right'],
 (4, 4): ['left', 'up', 'right', 'down'],
 (4, 5): ['left', 'up', 'down'],
 (4, 6): [],
 (5, 0): ['up', 'right', 'down'],
 (5, 1): ['left', 'right', 'down'],
 (5, 2): ['left', 'up'],
 (5, 3): [],
 (5, 4): ['up', 'right', 'down'],
 (5, 5): ['left', 'up', 'right'

In [206]:
net_maze = Network(heading="Lab 4 Maze",
                bgcolor ="#242020",
                font_color = "white",
                height = "750px",
                width = "100%" 
)
nodeColors={
    "wall":"red",
    "path": "White",
    "Goal": "Green",
    "Start": "Yellow",
    "Enemy": "Orange"
}
nodeColorsList=[]

for node in mazeWorldGraph.origin.keys():
    if mainMaze[node[0],node[1]]==1:
        nodeColorsList.append(nodeColors["path"])
    elif mainMaze[node[0],node[1]]==0:
        nodeColorsList.append(nodeColors["wall"])
    else:
        nodeColorsList.append(nodeColors["Enemy"])

nodes=["-".join(str(item) for item in el) for el in mazeWorldGraph.origin.keys()]

x_coords = []
y_coords = []

for node in mazeWorldGraph.origin.keys():
    x,y=mazeWorldGraph.getLocation(node)
    x_coords.append(x)
    y_coords.append(y)

sizes=[10]*len(nodes)
net_maze.add_nodes(nodes, color=nodeColorsList, x=x_coords, y=y_coords, size=sizes, title=nodes)

for node in net_maze.nodes:
    node['label']=''

edge_weights = {(intTupleTostr(k), intTupleTostr(v2)) : k2 for k, v in mazeWorldGraph.origin.items() for k2, v2 in v.items()}

edges=[]

for node_source in mazeWorldGraph.nodes():
    for node_target, action in mazeWorldGraph.get(node_source).items():
        #node_target or node_source is a tuple -> convert to str
        if (intTupleTostr(node_source),intTupleTostr(node_target)) not in edges and (intTupleTostr(node_target), intTupleTostr(node_source)):
            net_maze.add_edge(intTupleTostr(node_source),intTupleTostr(node_target), label=edge_weights[(intTupleTostr(node_source),intTupleTostr(node_target))])
            edges.append((intTupleTostr(node_source),intTupleTostr(node_target)))

for node in net_maze.nodes:
    if node['id']==intTupleTostr(goalState):
        node['color']=nodeColors['Goal']
    elif node['id']==intTupleTostr(initState):
        node['color']=nodeColors['Start']


In [207]:
net_maze.toggle_physics(False)

In [208]:
mp1=MazeProblem(initState, goalState, mazeWorldGraph)

In [209]:
BFSAgent=BestFirstSearchAgentProgram()
DLS=IDSearchAgentProgram()

In [210]:
BFSmazeAgent1=ProblemSolvingMazeAgentBFS(initState, mazeWorldGraph, goalState)
DLSAgent1=ProblemSolvingMazeAgentIDS(initState, mazeWorldGraph, goalState)

In [211]:
maze_Env1=Lab4NavEnvironment(mazeWorldGraph, mainMaze)
maze_Env2=Lab4NavEnvironment(mazeWorldGraph, mainMaze)

#maze_Env1=MazeNavigationEnvironment(mazeWorldGraph)
#maze_Env2=MazeNavigationEnvironment(mazeWorldGraph)

In [212]:
maze_Env1.add_thing(BFSmazeAgent1)
#if it runs X times change a wall to a path

The <Node (5, 0)> is being pushed to frontier ...
The <Node (5, 0)> is being extracted from frontier ...
The child <Node (4, 0)> is being pushed to frontier ...
The child <Node (5, 1)> is being pushed to frontier ...
The child <Node (6, 0)> is being pushed to frontier ...
The <Node (4, 0)> is being extracted from frontier ...
The child <Node (3, 0)> is being pushed to frontier ...
The <Node (3, 0)> is being extracted from frontier ...
The child <Node (2, 0)> is being pushed to frontier ...
The <Node (2, 0)> is being extracted from frontier ...
The child <Node (1, 0)> is being pushed to frontier ...
The child <Node (2, 1)> is being pushed to frontier ...
The <Node (1, 0)> is being extracted from frontier ...
The child <Node (0, 0)> is being pushed to frontier ...
The <Node (0, 0)> is being extracted from frontier ...
The <Node (2, 1)> is being extracted from frontier ...
The <Node (5, 1)> is being extracted from frontier ...
The child <Node (5, 2)> is being pushed to frontier ...
The ch

In [213]:
maze_Env1.run()

step 1:
Agent decided to do right.
Agent in (5, 1) with performance = 17
step 2:
Agent decided to do right.
Agent in (5, 2) with performance = 16
step 3:
Agent decided to do up.
Agent in (4, 2) with performance = 15
step 4:
Agent decided to do right.
Agent in (4, 3) with performance = 14
step 5:
Agent decided to do right.
Agent in (4, 4) with performance = 13
step 6:
Agent decided to do up.
Agent in (3, 4) with performance = 12
step 7:
Agent decided to do up.
Agent in (2, 4) with performance = 11
step 8:
Agent decided to do up.
Agent in (1, 4) with performance = 10
Agent in (1, 4) is under attack!
step 9:
Agent decided to do right.
Agent in (1, 5) with performance = 9.0
step 10:
Agent decided to do right.
Agent in (1, 6) with performance = 8.0
Agent reached all goals


In [214]:
maze_Env2.add_thing(DLSAgent1)

Solution (a sequence of actions) from the initial state to a goal: ['right', 'right', 'up', 'right', 'right', 'up', 'up', 'up', 'right', 'right']
The Agent in (5, 0) with performance 17


In [215]:
maze_Env2.run()

step 1:
Agent decided to do right.
Agent in (5, 1) with performance = 17
step 2:
Agent decided to do right.
Agent in (5, 2) with performance = 16
step 3:
Agent decided to do up.
Agent in (4, 2) with performance = 15
step 4:
Agent decided to do right.
Agent in (4, 3) with performance = 14
step 5:
Agent decided to do right.
Agent in (4, 4) with performance = 13
step 6:
Agent decided to do up.
Agent in (3, 4) with performance = 12
step 7:
Agent decided to do up.
Agent in (2, 4) with performance = 11
step 8:
Agent decided to do up.
Agent in (1, 4) with performance = 10
Agent in (1, 4) is under attack!
step 9:
Agent decided to do right.
Agent in (1, 5) with performance = 9.0
step 10:
Agent decided to do right.
Agent in (1, 6) with performance = 8.0
Agent reached all goals


In [216]:
BFSmazeAgent1.performance
DLSAgent1.performance

#print(BFSAP)
#print(DLSAP)

if BFSmazeAgent1.performance > DLSAgent1.performance:
    print("Uniform Cost Search Wins!")
elif DLSAgent1.performance > BFSmazeAgent1.performance:
    print("Iterative DLS Wins!")
else:
    print("It's a Tie!")

It's a Tie!


In [217]:
net_maze.show("graphMaze1.html", notebook=False)

graphMaze1.html
